In [3]:
import numpy as np #linear algebra
import pandas as pd #data preprocessing
import matplotlib.pyplot as plt #data visualization
import h5py
import PIL

from utils.general_utils import populate_breeds, get_imgMatrix_from_id, get_breed_value_from_id, get_filename_from_id
#other built utilities

#using inception_v3 to classify dog breeds
import tensorflow as tf #import tensroflow

In [4]:
inception_v3 = tf.keras.applications.InceptionV3() #create an instance of the inception_v3 model

In [5]:
%matplotlib inline

#import csv files
train = pd.read_csv("../data/included/labels.csv")
test = pd.read_csv("../data/included/test_id.csv")


In [6]:
#preprocess images
img = PIL.Image.open(get_filename_from_id(train.id[1],'../data/included/Train', filetype='.jpg'))
tf.keras.preprocessing.image.img_to_array(img).shape

(375, 500, 3)

In [7]:
#preprocess images - convert from cv2 image to numpy array
import cv2

fn = get_filename_from_id(train.id[1],'../data/included/Train', filetype='.jpg')
image = cv2.imread(fn)
tf.keras.preprocessing.image.img_to_array(img).shape

(375, 500, 3)

In [41]:
#preprocess images
img_matricies_list = []

for index, row in train.iterrows():
    fn = get_filename_from_id(row['id'],'../data/included/Train', filetype='.jpg')
    print(fn)
    img = cv2.imread(fn)
    img_matricies_list.append(tf.keras.preprocessing.image.img_to_array(img))
    if(index>=10):
        break

../data/included/Train/000bec180eb18c7604dcecc8fe0dba07.jpg
../data/included/Train/001513dfcb2ffafc82cccf4d8bbaba97.jpg
../data/included/Train/001cdf01b096e06d78e9e5112d419397.jpg
../data/included/Train/00214f311d5d2247d5dfe4fe24b2303d.jpg
../data/included/Train/0021f9ceb3235effd7fcde7f7538ed62.jpg
../data/included/Train/002211c81b498ef88e1b40b9abf84e1d.jpg
../data/included/Train/00290d3e1fdd27226ba27a8ce248ce85.jpg
../data/included/Train/002a283a315af96eaea0e28e7163b21b.jpg
../data/included/Train/003df8b8a8b05244b1d920bb6cf451f9.jpg
../data/included/Train/0042188c895a2f14ef64a918ed9c7b64.jpg
../data/included/Train/004396df1acd0f1247b740ca2b14616e.jpg


In [50]:
img_matricies_list[10].shape

(500, 375, 3)

In [54]:
img_matricies_np = np.asarray(img_matricies_list) #convert the list of matricies to a numpy array
len(img_matricies_np)
img_matricies_np[1].shape

(375, 500, 3)

In [55]:
#prepare the breed list dataframe AKA output information

BREED_LIST = "../data/preprocessed_data/breed_list.csv"
labels = populate_breeds(BREED_LIST) #get the list of all dog breeds
labels_np = np.array(labels).reshape(120,1) #labels list reshaped to numpy array
labels_df = pd.DataFrame(labels_np ,columns = list("B")) #convert list of labels (dog breeds)
#into a dataframe for eaiser use with classifier
#column name is 'B' because I'm not sure how to make it more than one letter

labels_np = labels_np.transpose()
target_values_list = []
print(train.loc[5]['id'])

#get_breed_matrix_from_id(train.loc[5]['id'], labels_list=labels)


#returns a list of 0.0s except for one index which corresponds to the location of that dog breed

002211c81b498ef88e1b40b9abf84e1d


In [59]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": img_matricies_np},
    y=labels_np,
    num_epochs=None,
    shuffle=True)


#create an input layer
#input = tf.placeholder(tf.float32, [None, image_size*image_size])
#labels = tf.placeholder(tf.float32, [None, labels_size])
#tf.Tensor = train_input_fn
inception_v3.get_input_at = train_input_fn



<bound method Model.train_on_batch of <tensorflow.python.keras._impl.keras.engine.training.Model object at 0x7fe8ec588fd0>>

In [65]:
inception_v3.build


<bound method Model.build of <tensorflow.python.keras._impl.keras.engine.training.Model object at 0x7fe8ec588fd0>>